In [173]:
# imports

import pandas as pd
import numpy as np
from os.path import join
import nltk
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [174]:
# read in data
# some of these lists are weird...and maybe shouldn't be used...
# or at least filtered by review and maybe genre to eliminate weird entries
path = '/Users/rmillin/Documents/Insight/diverse-reading/data'
fname = 'diverse_2_01_25.json'
data = pd.read_json(join(path,fname), orient='records')
data

# The descriptions are screwed up - parts of them repeat.  I'm not sure what to do about this.  
# If first reducing to words, could live with it and use unique words.

,ISBN,author,avg_ratings,awards,character_names,description,genres,language,num_pages,num_ratings,num_reviews,places,publish_date,title,url
0,NaN,Langston Hughes,4.43,[],[],[A beautiful new edition of this beloved poet'...,"[Poetry, Classics, Cultural, African American,...",English,NaN,459,47,[],1945-10-11 00:00:00,The Weary Blues,https://www.goodreads.com/book/show/15762561-t...
1,0435902296,Thomas Mofolo,3.70,[],[],[Tells the classic story of the Zulu hero Chaka.],"[Cultural, Africa, Fiction, Southern Africa, L...",English,168.0,330,39,[],1983-01-01 00:00:00,Chaka,https://www.goodreads.com/book/show/1101837.Chaka
2,1555530664,Jessie Redmon Fauset,3.77,[],[],"[Set in Philadelphia some 60 years ago, There ...","[Fiction, Classics, Cultural, African American...",English,297.0,125,9,[],1989-11-07 00:00:00,There Is Confusion,https://www.goodreads.com/book/show/227986.The...
3,0870113623,Naoya Shiga,3.75,[],[],[Tells the story of a young man's passage thro...,"[Cultural, Japan, Asian Literature, Japanese L...",English,408.0,142,15,[],1979-12-01 00:00:00,A Dark Night's Passing (Japan's Modern Writers),https://www.goodreads.com/book/show/1396965.A_...
4,0826328180,Gabriela Mistral,4.17,[],[],[The first Nobel Prize in literature to be awa...,"[Poetry, Nobel Prize, European Literature, Spa...",English,407.0,131,6,[],2003-12-02 00:00:00,Selected Poems,https://www.goodreads.com/book/show/67998.Sele...
5,0252030923,Georgia Douglas Johnson,3.33,[],[],[This volume collects twelve of Georgia Dougla...,"[Cultural, African American, Drama]",English,216.0,3,0,[],2006-04-03 00:00:00,The Plays of Georgia Douglas Johnson: From the...,https://www.goodreads.com/book/show/3567389-th...
6,0803299176,Zitkala-Ša,3.88,[],[],"[, first published in 1921, is a collection of...","[Short Stories, Fiction, Classics]",English,196.0,635,48,[],2003-09-01 00:00:00,American Indian Stories,https://www.goodreads.com/book/show/192424.Ame...
7,0385412959,Countee Cullen,4.26,[],[],"[For the first time in more than two decades, ...","[Poetry, Classics, The United States Of Americ...",English,618.0,58,6,[],1990-12-01 00:00:00,My Soul's High Song: The Collected Writings,https://www.goodreads.com/book/show/873428.My_...
8,068481580X,Wallace Thurman,3.95,[],[],[One of the most widely read and controversial...,"[Fiction, Cultural, African American, Classics...",English,221.0,"1,568",142,"[Idaho, New York City, New York]",1996-02-02 00:00:00,The Blacker the Berry...,https://www.goodreads.com/book/show/868440.The...
9,0803293321,Luther Standing Bear,4.00,[],[],"[When it was first published in 1928, Luther S...","[History, Nonfiction, Biography, Autobiography...",English,296.0,302,24,[],2006-11-01 00:00:00,My People the Sioux,https://www.goodreads.com/book/show/1074976.My...


In [175]:
# clean summaries
# start at the last element.  If that element contains a preceding element, delete the preceding one.

def clean_summary(summary):
    if isinstance(summary,list):
        ind2 = 0
        ind1=len(summary)-1
        while ind1>=0 and ind2<len(summary):
            while ind2<ind1:
                if summary[ind2] in summary[ind1]:
                    summary.remove(summary[ind2])
                    ind1-=1
                else:
                    ind2+=1
            ind1-=1
            ind2=0
    return summary

In [176]:
# run the cleaning function on a dataset

data['description'] = data['description'].apply(clean_summary)

In [177]:
# flatten summary lists into strings
data['description'] = data['description'].apply(lambda x: "".join(x) if isinstance(x,list) else str(x))


In [178]:
data.loc[4,'description']

"The first Nobel Prize in literature to be awarded to a Latin American writer went to the Chilean poet Gabriela Mistral. Famous and beloved during her lifetime all over Latin America and in Europe, Mistral has never been known in North America as she deserves to be. The reputation of her more flamboyant and accessible friend and countryman Pablo Neruda has overshadowed hers, and she has been officially sentimentalised into a 'poetess' of children and motherhood. Translations, and even selections of her work in Spanish, have tended to underplay the darkness, the strangeness, and the raging intensity of her poems of grief and pain, the yearning power of her evocations of the Chilean landscape, the stark music of her Round Dances, the visionary splendour of her Hymns of America. During her lifetime Mistral published four books: Desolation, Tenderness, Clearcut, and Winepress. These are included in the 'Complete' Nobel edition published in Madrid; the Poem of Chile, her last book, was prin

In [179]:
def tokenize_text(word_tokens):
    return [w for w in word_tokens if not w in stop_words]    

def lemmatize_text(word_tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in word_tokens]



stop_words = set(stopwords.words('english'))

# convert to lower case
data['description'] = data['description'].apply(lambda x:x.lower())
print(data.loc[0,'description'])

# eliminate numbers
data['description'] = data['description'].apply(lambda x:re.sub("[\d]", " ",  x))
print(data.loc[0,'description'])

# tokenize
data['description'] = data['description'].apply(lambda x:re.sub("[^\w]", " ",  x).split())
print(data.loc[0,'description'])

# remove stopwords
data['description'] = data['description'].apply(lambda x:tokenize_text(x))
print(data.loc[0,'description'])

# lemmatize
data['description'] = data['description'].apply(lambda x:lemmatize_text(x))



a beautiful new edition of this beloved poet's first collection, originally published in 1926 when he was just twenty-four. from the opening "proem" (prologue poem) he offers in this first book-"i am a negro: / black as night is black, / black the depths of my africa"-hughes spoke directly, intimately, and powerfully of the experiences of african americans, at a time when their voices were newly being heard in our literature. as his knopf editor carl van vechten wrote in a brief introduction to the original 1926 volume, illuminating the potential of this promising young voice, "his cabaret songs throb with the true jazz rhythm; his sea-pieces ache with a calm, melancholy lyricism; he cries bitterly from the heart of his race...always, however, his stanzas are subjective, personal" and, he concludes, they are "the expression [of] an essentially sensitive and subtly illusive nature." that illusive nature darts among these early lines and begins to reveal itself, sometimes with shocking c

In [181]:
data.loc[0,'description']

['beautiful',
 'new',
 'edition',
 'beloved',
 'poet',
 'first',
 'collection',
 'originally',
 'published',
 'twenty',
 'four',
 'opening',
 'proem',
 'prologue',
 'poem',
 'offer',
 'first',
 'book',
 'negro',
 'black',
 'night',
 'black',
 'black',
 'depth',
 'africa',
 'hughes',
 'spoke',
 'directly',
 'intimately',
 'powerfully',
 'experience',
 'african',
 'american',
 'time',
 'voice',
 'newly',
 'heard',
 'literature',
 'knopf',
 'editor',
 'carl',
 'van',
 'vechten',
 'wrote',
 'brief',
 'introduction',
 'original',
 'volume',
 'illuminating',
 'potential',
 'promising',
 'young',
 'voice',
 'cabaret',
 'song',
 'throb',
 'true',
 'jazz',
 'rhythm',
 'sea',
 'piece',
 'ache',
 'calm',
 'melancholy',
 'lyricism',
 'cry',
 'bitterly',
 'heart',
 'race',
 'always',
 'however',
 'stanza',
 'subjective',
 'personal',
 'concludes',
 'expression',
 'essentially',
 'sensitive',
 'subtly',
 'illusive',
 'nature',
 'illusive',
 'nature',
 'dart',
 'among',
 'early',
 'line',
 'begin',
 

In [180]:
data['description']

0     [beautiful, new, edition, beloved, poet, first...
1             [tell, classic, story, zulu, hero, chaka]
2     [set, philadelphia, year, ago, confusion, trac...
3     [tell, story, young, man, passage, sequence, d...
4     [first, nobel, prize, literature, awarded, lat...
5     [volume, collect, twelve, georgia, douglas, jo...
6     [first, published, collection, childhood, stor...
7     [first, time, two, decade, best, countee, cull...
8     [one, widely, read, controversial, work, harle...
9     [first, published, luther, standing, bear, aut...
10    [one, handful, novel, published, black, woman,...
11    [first, novel, noted, black, novelist, folklor...
12    [charm, humor, deep, understanding, japanese, ...
13    [seaside, bosque, de, mar, guest, hotel, centr...
14    [nobel, laureate, first, novel, spotlight, sma...
15    [set, brooklyn, depression, world, war, ii, br...
16    [rebelling, constraint, family, society, young...
17    [classic, volume, distinguished, modern, p